<a href="https://colab.research.google.com/github/anandlo/NLP-with-BERT/blob/main/NLP_Bert_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pandas as pd

######read csv file
df1 = pd.read_csv('https://query.data.world/s/e6p63p3nbskiwiy3okh6umyizdc4d4',engine='python',encoding='ISO-8859-1')


In [ ]:
df1.head()
df1.shape

In [ ]:
## Here, we have taken only first 2000 records because of the limited Ram of free version of Google Colab
df2=df1.iloc[:2000,:]
df=df2.copy()
print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df['existence'].isnull().sum()

In [ ]:
# See the number of records for the output label. Here, we see that In some records "Y" is inplace of "Yes" and "N" in place of "No"
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['existence'].value_counts()


In [ ]:
 #Here, we are replacing "Y" with "Yes" and "N" with "No"
 df['existence'] = df['existence'].apply(lambda x: 'Yes' if (x=='Y')|(x=='Yes') else 'No' if (x=='N')|(x=='No') else 'Nil')
 df.head()

In [ ]:
df['existence'].value_counts()

In [ ]:
## Labels are encoding to 0 or 1
df['existence']=le.fit_transform(df['existence'])

In [ ]:
df['existence'].value_counts()

In [ ]:
###installation of transformers

!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
##Divide the input into tokens using BERT Tokenizer

tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
###The BERT model receives a fixed length of sentence as input. Usually the maximum length of a sentence depends on the data we are working on.
###For sentences that are shorter than this maximum length, we will have to add paddings (empty tokens) to the sentences to make up the length
###Here, we are adding zero to make up the length

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

(2000, 61)

In [ ]:
##Prepare the attention Mask by relacing all the numbers by 1 except 0

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 61)

In [ ]:
##Convert the padded input and attention mask into tensor and provide both as input to the model(.DistilBertModel)
### the model will provide you the last hidden state as output

input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
##extract the features from the last hidden state

features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df['existence']

In [ ]:
## Split the dataset into train and test data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
##Apply logistic regression model

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
##Find the score

lr_clf.score(test_features, test_labels)

0.81

In [ ]:
##Take s taring to predict
review_text = "Global warming kills environment and trees"

In [ ]:
## Prepare the input in the format acceptable by the model

tokenized = tokenizer.encode(review_text, add_special_tokens=True)

padded = np.array([tokenized + [0]*(max_len-len(tokenized)) ])
np.array(padded).shape
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
print(input_ids)
print(attention_mask)

tensor([[  101,  3795, 12959,  8563,  4044,  1998,  3628,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()


In [ ]:
##Predict the label for the string
print(lr_clf.predict(features))

[2]
